In [27]:
from gensim.models import KeyedVectors
from torch.utils.data import DataLoader
from dataset_structure import SegmentationDataset
from model_structure import LstmCrf
import torch
from nltk.tokenize import sent_tokenize
import unicodedata
from aux_functions import *

Tentativa de segmentar o DODF *data_path* usando o modelo de avisos de licitacao *model_path*

In [ ]:
data_path = "../dodf_txt/2019/03_Março/DODF 043 01-03-2019.txt"
model_path = "../Models/seg_model_256_aviso_de_licitacao"

In [28]:
#Lê o dodf, junta o texto em um parágrafo só e tokeniza em sentenças
data = open(data_path, "rt").read()
text = join_block(data)
sents = sent_tokenize(text)

In [29]:
#cria tags IOB para as sentenças e escreve o resultado em um arquivo
# (isso é feito só pra poder usar as mesmas funções, as tags não são usadas)

output = open("../Processed_datasets/production_test.iob", "w")

iob_sents = ["O " + x for x in sents]
output.write("\n".join(iob_sents))


689030

In [26]:
#carrega o embedding pre treinado e os dicionários salvos durante o treinamento
emb = KeyedVectors.load_word2vec_format("../Models/cbow_s50_2.txt")
word2idx = {}
for j in emb.index_to_key:
    word = (
        unicodedata.normalize("NFKD", j)
        .encode("ascii", "ignore")
        .decode("utf8")
    )
    word2idx[word] = emb.key_to_index[j]

tag2idx = {"B": 0, "I": 1, "O": 2}
idx2tag = {0: "B", 1: "I", 2: "O"}


In [30]:
#carrega o dodf usando a classe SegmentationDataset, que processa o texto para a classificação
ata = SegmentationDataset(
    tag2idx=tag2idx,
    word2idx=word2idx,
    set_type="whole",
    path="../Processed_datasets/production_test.iob",
)

data_loader = DataLoader(
    dataset=data, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)

In [31]:
#carrega o modelo treinado usando o dicionário de estados salvo no treinamento

model = LstmCrf(
    embedding_dim=50,
    num_tags=3,
    hidden_dim=256,
    pretrained_emb=emb,
    idx2tag=idx2tag,
    path=model_path,
)
model = model.to(model.device)

model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [32]:
#faz a predição
x, mask, y = data.x, data.mask, data.y
pred = model.predict(x.to(model.device), mask.to(model.device))

/home/munak98/.local/lib/python3.8/site-packages/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  ../aten/src/ATen/native/TensorCompare.cpp:328.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


In [ ]:
#seleciona os atos que foram identificados
j = 0
findings = []
pred = [item for sublist in pred for item in sublist]
text = [" ".join(item) for sublist in data.truncated for item in sublist]

while j < len(pred):
    if idx2tag[pred[j]] == 'B':
        new_finding = [text[j]]
        j += 1
        while idx2tag[pred[j]] == 'I':
            new_finding.append(text[j])
            j += 1
        findings.append(" ".join(new_finding))
    else:
        j += 1

In [33]:
for act in findings:
    print(act, "\n")